## Import Donor dataset

In [1]:
import pandas as pd
import random

# Import the csv file to be used for this exercise
donor = pd.read_csv("donor_fake.csv")
donor

,Firstname,Lastname,Sex,Address,Postcode
0,Hayley,Reichel-Watsica,Male,"1 Boyce'S Ave, Clifton, Bristol",BS8 4AA
1,Haylie,Macejkovic,Male,"Gas Ferry Rd, Bristol",BS1 6TY
2,Vernie,Lesch,Male,"59 Broad St, Bristol",BS1 2EJ
3,Austyn,Turner,Female,"23 Small St, Bristol",BS1 1DW
4,Bartholome,Jerde,Male,"29 Victoria St, Redcliffe, Bristol",BS1 6AA
...,...,...,...,...,...
193,Mariah,Schinner,Male,"148 Gloucester Rd, Bishopston, Avon, Bristol",BS7 8NU
194,Lazaro,Murphy,Male,"8 Fairfax St, Bristol",BS1 3DB
195,Bria,Rowe,Female,"39 High St, Bristol",BS16 5HN
196,Dayna,Feest,Female,"93 Whiteladies Rd, Clifton, Bristol",BS8 2PN


## Data Tranformation

#### Add random five donation preference to each donor

In [2]:
# List of 10 unique text items
texts = ["Poverty", "Education", "Healthcare", "Women", "Elderly Care"]

# Number of rows and columns for preferences
num_rows = len(donor)
num_columns = 5
new_columns = [f'Pref_{i+1}' for i in range(num_columns)]

# Add new columns with randomized text ensuring no matching text within any row
for i in range(num_rows):
    row_texts = random.sample(texts, num_columns)  # Randomly select 5 unique texts for each row
    for j, col in enumerate(new_columns):
        donor.at[i, col] = row_texts[j]
        

#### Generate Latitude and Longitude from Postcode

In [3]:
from geopy.geocoders import GoogleV3
from geopy.exc import GeocoderTimedOut
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize the geolocator with your Google Maps API key
api_key = os.getenv('google_map_api_key')
geolocator = GoogleV3(api_key=api_key)

# Function to geocode postal codes
def geocode_postcode(postcode):
    try:
        location = geolocator.geocode(postcode, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None

# Apply the geocoding function to the DataFrame
donor['Latitude'], donor['Longitude'] = zip(*donor['Postcode'].apply(geocode_postcode))

In [4]:
donor.head()

,Firstname,Lastname,Sex,Address,Postcode,Pref_1,Pref_2,Pref_3,Pref_4,Pref_5,Latitude,Longitude
0,Hayley,Reichel-Watsica,Male,"1 Boyce'S Ave, Clifton, Bristol",BS8 4AA,Poverty,Education,Healthcare,Elderly Care,Women,51.455353,-2.618385
1,Haylie,Macejkovic,Male,"Gas Ferry Rd, Bristol",BS1 6TY,Healthcare,Women,Elderly Care,Education,Poverty,51.449247,-2.608013
2,Vernie,Lesch,Male,"59 Broad St, Bristol",BS1 2EJ,Elderly Care,Women,Education,Poverty,Healthcare,51.455169,-2.593152
3,Austyn,Turner,Female,"23 Small St, Bristol",BS1 1DW,Elderly Care,Healthcare,Women,Poverty,Education,51.454848,-2.594342
4,Bartholome,Jerde,Male,"29 Victoria St, Redcliffe, Bristol",BS1 6AA,Education,Poverty,Women,Healthcare,Elderly Care,51.452695,-2.589709


In [5]:
# Check for NaN or empty values
print("\nCheck for NaN or empty values:")
print(donor.isna())  # Check for NaN
print(donor.isna().any(axis=1))  # Check for NaN in any column
print(donor.isin(['', ' ', None]).any(axis=1))  # Check for empty strings or None


Check for NaN or empty values:
     Firstname  Lastname    Sex  Address  Postcode  Pref_1  Pref_2  Pref_3  \
0        False     False  False    False     False   False   False   False   
1        False     False  False    False     False   False   False   False   
2        False     False  False    False     False   False   False   False   
3        False     False  False    False     False   False   False   False   
4        False     False  False    False     False   False   False   False   
..         ...       ...    ...      ...       ...     ...     ...     ...   
193      False     False  False    False     False   False   False   False   
194      False     False  False    False     False   False   False   False   
195      False     False  False    False     False   False   False   False   
196      False     False  False    False     False   False   False   False   
197      False     False  False    False     False   False   False   False   

     Pref_4  Pref_5  Latitude  

In [6]:
# Drop rows with NaN or empty values
donor_cleaned = donor.dropna()  # Drop rows with any NaN values
donor_cleaned = donor_cleaned[~donor_cleaned.isin(['', ' ','nan', None]).any(axis=1)]  # Drop rows with any empty strings or None

#### Generate fake email address for each donor

In [7]:
from faker import Faker

# Initialize Faker generator
fake = Faker()

# Function to generate random email using first name and last name
def generate_random_email(first_name, last_name):
    # Generate email using first name and last name
    email = f"{first_name.lower()}.{last_name.lower()}@gmail.com"  # Replace example.com with your domain
    return email

# Apply the function to generate email for each row
donor_cleaned['Email'] = donor_cleaned.apply(lambda row: generate_random_email(row['Firstname'], row['Lastname']), axis=1)

In [8]:
# Add an index column starting from 1
donor_cleaned.reset_index(drop=True, inplace=True)
donor_cleaned.index = donor_cleaned.index + 1
donor_cleaned.index.name = 'Id'

### Preview transformed donor data

In [9]:
# Display the DataFrame to verify
donor_cleaned.head()

,Firstname,Lastname,Sex,Address,Postcode,Pref_1,Pref_2,Pref_3,Pref_4,Pref_5,Latitude,Longitude,Email
Id,,,,,,,,,,,,,
1,Hayley,Reichel-Watsica,Male,"1 Boyce'S Ave, Clifton, Bristol",BS8 4AA,Poverty,Education,Healthcare,Elderly Care,Women,51.455353,-2.618385,hayley.reichel-watsica@gmail.com
2,Haylie,Macejkovic,Male,"Gas Ferry Rd, Bristol",BS1 6TY,Healthcare,Women,Elderly Care,Education,Poverty,51.449247,-2.608013,haylie.macejkovic@gmail.com
3,Vernie,Lesch,Male,"59 Broad St, Bristol",BS1 2EJ,Elderly Care,Women,Education,Poverty,Healthcare,51.455169,-2.593152,vernie.lesch@gmail.com
4,Austyn,Turner,Female,"23 Small St, Bristol",BS1 1DW,Elderly Care,Healthcare,Women,Poverty,Education,51.454848,-2.594342,austyn.turner@gmail.com
5,Bartholome,Jerde,Male,"29 Victoria St, Redcliffe, Bristol",BS1 6AA,Education,Poverty,Women,Healthcare,Elderly Care,51.452695,-2.589709,bartholome.jerde@gmail.com


## Export transformed donor data

In [10]:
# Save to a new CSV file
donor_cleaned.to_csv('sample_donor.csv', index=True)